<div style="background-color:red; height: 3em">

</div>

# Block description: `Backtesting`

<div style="background-color:red; height: 3em">

</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import sys
import time
import json
import requests
from pandas.io.json import json_normalize
import csv

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# import the data
df = pd.read_csv('../data/stock_infos/dividend_values_per_symbol.csv')
stock_df = pd.read_csv('../data/stock_infos/dividend_values_per_symbol.csv')
df

,date,variable,value,symbol
0,2005-03,adjDividend,0.0065,ADBE
1,2004-12,adjDividend,0.0065,ADBE
2,2004-09,adjDividend,0.0065,ADBE
3,2004-06,adjDividend,0.0065,ADBE
4,2004-03,adjDividend,0.0065,ADBE
...,...,...,...,...
101225,1988-02,declarationDate,1987-12-17,CMCSA
101226,1987-11,declarationDate,1987-09-18,CMCSA
101227,1987-08,declarationDate,1987-06-26,CMCSA
101228,1987-05,declarationDate,1987-03-27,CMCSA


In [8]:
# split df by symbol and then add the splitted values as columns
pivot_df = df.pivot_table(index=['date', 'variable'], columns='symbol', values='value', aggfunc='first')


def see_splited_dfs(splitted_df, result_df):
    # print(splitted_df)
    
    return splitted_df

result_df = pd.DataFrame()


dfs = df.groupby("symbol").apply(lambda x: see_splited_dfs(x, result_df))

df.pivot_table(index=['date', 'variable'], columns='symbol', values='value', aggfunc='first').reset_index().to_csv('../data/stock_infos/dividend_values_per_symbol_pivoted.csv', index=False)

In [4]:
import pandas as pd

# Sample data
data = {'date': ['20.01', '20.01', '21.01'],
        'variable': ['adj_clo', 'adj_open', 'adj_clo'],
        'symbol': ['appl', 'appl', 'msft'],
        'value': [10, 12, 21]}

# Create a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
pivot_df = df.pivot_table(index=['date', 'variable'], columns='symbol', values='value', aggfunc='first')

# Reset index to make 'date' and 'variable' regular columns
pivot_df.reset_index(inplace=True)

# Fill NaN values with 0 (if needed)
pivot_df = pivot_df.fillna(0)

# Print the result
print(pivot_df)


symbol   date  variable  appl  msft
0       20.01   adj_clo  10.0   0.0
1       20.01  adj_open  12.0   0.0
2       21.01   adj_clo   0.0  21.0


In [4]:
df[(df["variable"] == "adjDividend") & (df["symbol"] == "MSFT")]

,date,variable,value,symbol
3405,2024-02,adjDividend,0.75,MSFT
3406,2023-11,adjDividend,0.75,MSFT
3407,2023-08,adjDividend,0.68,MSFT
3408,2023-05,adjDividend,0.68,MSFT
3409,2023-02,adjDividend,0.68,MSFT
...,...,...,...,...
3481,2005-02,adjDividend,0.08,MSFT
3482,2004-11,adjDividend,3.08,MSFT
3483,2004-08,adjDividend,0.08,MSFT
3484,2003-10,adjDividend,0.16,MSFT


In [10]:
# filtering data to get symbols which haven't lost there dividenden until a given date

def check_for_yearly_dividend(df: pd.DataFrame, x: datetime.date, look_back_years: int = 5):
    """
    Checks if a symbol has a dividend for every year in the look_back_years until the given date x

    Args:
        df (pd.DataFrame): dataframe with dividend values
        x (datetime.date): date to check for
        look_back_years (int, optional): look back years. Defaults to 5.
    """
    df_temp = df.loc[(df["variable"] == "adjDividend") & (pd.to_datetime(df["date"]) <= pd.to_datetime(x))]
    df_temp["year_period"] = pd.to_datetime(df["date"]).dt.to_period("Y")
    
    df_temp = df_temp.groupby("year_period").count().reset_index()
    
    # subtract year of x with look_back_years
    first_year = x.year - look_back_years

    result = df_temp.loc[df_temp["year_period"].astype(str).astype(int) >= first_year]["value"].all()

    return result


check_for_yearly_dividend(df[df["symbol"] == "MSFT"], datetime.date(2020, 1, 1), 5)



C:\Users\Bene\AppData\Local\Temp/ipykernel_38076/4163765803.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["year_period"] = pd.to_datetime(df["date"]).dt.to_period("Y")


True

In [40]:
smth = df

smth["year_period"] = pd.to_datetime(df["date"]).dt.to_period("Y")
smth = smth.loc[(df["variable"] == "adjDividend") & (smth["symbol"] == "MSFT")]
smth

,date,variable,value,symbol,year_period
3405,2024-02,adjDividend,0.75,MSFT,2024
3406,2023-11,adjDividend,0.75,MSFT,2023
3407,2023-08,adjDividend,0.68,MSFT,2023
3408,2023-05,adjDividend,0.68,MSFT,2023
3409,2023-02,adjDividend,0.68,MSFT,2023
...,...,...,...,...,...
3481,2005-02,adjDividend,0.08,MSFT,2005
3482,2004-11,adjDividend,3.08,MSFT,2004
3483,2004-08,adjDividend,0.08,MSFT,2004
3484,2003-10,adjDividend,0.16,MSFT,2003


In [36]:
# filtering data to get symbols wich have dividend over 5

def check_for_min_dividend(df: pd.DataFrame, min_dividend: float = 0.05, x: datetime.date = datetime.date(2020, 1, 1), look_back_years: int = 5):
    """
    Checks if a symbol has a dividend for every year in the look_back_years until the given date x
    
    Args:
        df (pd.DataFrame): dataframe with dividend values
        min_dividend (float, optional): minimum dividend. Defaults to 5.
    """

    # TODO think about the amout of times a dividend is paid in a year
    df_temp = df.loc[(df["variable"] == "adjDividend") & (pd.to_datetime(df["date"]) <= pd.to_datetime(x))]
    df_temp["year_period"] = pd.to_datetime(df["date"]).dt.to_period("Y")
    
    df_temp["value"] = df_temp["value"].astype(float)
    df_temp = df_temp.groupby("year_period").sum().reset_index()
    
    # subtract year of x with look_back_years
    first_year = x.year - look_back_years

    # df_temp["year_period"] = df_temp["year_period"].astype(str).astype(int)
    print(df_temp["value"])
    df_temp = df_temp[df_temp["year_period"].astype(str).astype(int) > first_year]

    result = df_temp.loc[df_temp["value"].astype(float) >= min_dividend]["value"].all()

    return result

check_for_min_dividend(df[df["symbol"] == "MSFT"], 0.05, datetime.date(2020, 1, 1), 15)

# filtering data to get symbols wich have consistent increased there dividends


0     0.24
1     3.16
2     0.32
3     0.37
4     0.41
5     0.46
6     0.52
7     0.55
8     0.68
9     0.83
10    0.97
11    1.15
12    1.29
13    1.47
14    1.59
15    1.72
16    1.89
Name: value, dtype: float64


C:\Users\Bene\AppData\Local\Temp/ipykernel_38076/2643244974.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["year_period"] = pd.to_datetime(df["date"]).dt.to_period("Y")
C:\Users\Bene\AppData\Local\Temp/ipykernel_38076/2643244974.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["value"] = df_temp["value"].astype(float)


True

In [43]:
166.16 * 0.0052

0.8640319999999999

In [ ]:
# calculate dividenden yield
# dividenden yield = dividend per share / price per share